## Notebook for playing around with variables in experimental and master objects

In [1]:
%run ./rob_setup_notebook.ipynb

import utils.interareal_analysis as ia
import utils.interareal_plotting as ip
import utils.utils_funcs as uf
from ia_funcs import *

from utils.paq2py import *


Suite2p path: ['/home/rlees/Documents/Code/suite2p/suite2p']
QNAP_path: /home/rlees/mnt/qnap 
QNAP_data_path /home/rlees/mnt/qnap/Data 
Pickle folder: /home/rlees/mnt/qnap/pkl_files 
Master pickle: /home/rlees/mnt/qnap/master_pkl/master_obj.pkl 
Figure save path: /home/rlees/mnt/qnap/Analysis/Plots 
STA movie save path: /home/rlees/mnt/qnap/Analysis/STA_movies 
S2 borders path: /home/rlees/mnt/qnap/Analysis/S2_borders


### Add or check attributes in all exp_obj pkl files

In [ ]:
pkl_files = os.listdir(pkl_folder)

for i,pkl in enumerate(pkl_files):
    if '.pkl' in pkl:
        
        pkl_path = os.path.join(pkl_folder, pkl)

        with open(pkl_path, 'rb') as f:
            exp_obj = pickle.load(f)
            
#             with open(pkl_path, 'wb') as f:
#                 pickle.dump(exp_obj, f)
                
#         pkl_list = [exp_obj.photostim_r, exp_obj.photostim_s]

#         if exp_obj.spont.n_frames > 0:
#             pkl_list.append(exp_obj.spont)

#         if exp_obj.whisker_stim.n_frames > 0:
#             pkl_list.append(exp_obj.whisker_stim)
        
#         for sub_obj in pkl_list:

### Play with data from individual experimental objects

In [242]:
# List of pickle object file names
# '2019-06-11_J048',
# '2019-06-13_J052',
# '2019-06-25_RL048',
# '2019-07-15_RL052',
# '2019-08-14_J058',
# '2019-08-14_J059',
# '2019-08-15_RL055',
# '2019-11-14_J061',
# '2019-11-14_J063',
# '2019-11-15_J064',
# '2019-11-18_J065',
# '2020-02-27_RL075',
# '2020-03-03_RL074',
# '2020-03-05_RL070',
# '2020-03-05_RL072',
# '2020-03-12_RL070',
# '2020-03-16_RL074',
# '2020-03-19_RL075',
# '2020-03-20_RL080'

pkl_path = os.path.join(pkl_folder, '2020-03-03_RL074.pkl')

with open(pkl_path, 'rb') as f:
    exp_obj = pickle.load(f)
    
print('Loaded pickled object.')

Loaded pickled object.


### Show all attributes in exp_obj or sub_objs

In [3]:
# Show all attributes in exp_obj or sub_objs
print('Session object attributes')
for key, value in vars(exp_obj).items():
    print(key)

print('\nExperimental object attributes')
for key, value in vars(exp_obj.photostim_r).items():
    print(key)

Session object attributes
ss_id
sheet_name
s2p_path
photostim_r
photostim_s
whisker_stim
spont
frame_list

Experimental object attributes
tiff_path
naparm_path
paq_path
sheet_name
s2p_path
stim_channel
stim_type
fps
frame_x
frame_y
n_planes
pix_sz_x
pix_sz_y
scan_x
scan_y
zoom
n_frames
last_good_frame
n_groups
n_reps
n_shots
n_trials
inter_point_delay
spiral_size
single_stim_dur
stim_dur
duration_frames
stim_start_frames
frames
cell_id
n_units
cell_plane
cell_med
cell_s1
cell_x
cell_y
raw
mean_img
xoff
yoff
pre_frames
post_frames
all_trials
stas
all_amplitudes
sta_amplitudes
t_tests
wilcoxons
sta_sig
sta_sig_nomulti
single_sig
n_targets
target_coords
target_areas
targeted_cells
n_targeted_cells


In [259]:
sub_obj = exp_obj.whisker_stim

total_frames = np.sum(exp_obj.frame_list)
dff = np.zeros([sub_obj.n_planes, sub_obj.n_units[0], total_frames])

for plane in range(sub_obj.n_planes):
    dff[plane] = uf.dfof2(sub_obj.raw[plane])
    
paq = paq_read(sub_obj.paq_path)
frame_clock = uf.paq_data(paq, 'frame_clock', threshold_ttl=True, plot=False)
stim_times = uf.paq_data(paq, sub_obj.stim_channel, threshold_ttl=True, plot=False)

sub_obj.stim_start_frames = []

for plane in range(sub_obj.n_planes):

    stim_start_frames = uf.stim_start_frame(paq, sub_obj.stim_channel, 
                        frame_clock, stim_times, plane, sub_obj.n_planes)
            
    sub_obj.stim_start_frames.append(stim_start_frames)

#correct this based on txt file
duration_ms = sub_obj.stim_dur
frame_rate = sub_obj.fps/sub_obj.n_planes
duration_frames = np.ceil((duration_ms/1000)*frame_rate) + 1 #+1 as sometimes artifact leaked in to data
sub_obj.duration_frames = int(duration_frames)
print('total stim duration (frames):', int(duration_frames))

total stim duration (frames): 17


In [261]:
if sub_obj.stim_start_frames:
        
    sub_obj.all_trials = []
    sub_obj.stas = []
    sub_obj.sta_amplitudes = []
    
    # key parameter for the sta, how many frames before and after the stim onset do you want to use
    sub_obj.pre_frames = int(np.ceil(sub_obj.fps*0.5)) # 500 ms pre-stim period
    sub_obj.post_frames = int(np.ceil(sub_obj.fps*3)) # 3000 ms post-stim period
    
    for plane in range(sub_obj.n_planes):

        plane_dff = dff[plane][:,sub_obj.frames]
        
        pre, post, _ = uf.test_responsive(plane_dff, frame_clock, stim_times, 
                                                 sub_obj.pre_frames, sub_obj.post_frames, 
                                                 sub_obj.duration_frames)
        
        # reconstruct trials from pre and post periods
        if pre.shape[1]/sub_obj.n_trials
        
        pre_trials = np.reshape(pre, (pre.shape[0], sub_obj.n_trials, -1))
        post_trials = np.reshape(post, (post.shape[0], sub_obj.n_trials, -1))
        stim_offset = np.zeros((pre.shape[0], sub_obj.n_trials, sub_obj.duration_frames))
        all_trials = np.concatenate((pre_trials, stim_offset, post_trials), axis=2)
        
        sub_obj.all_trials.append(all_trials)
        
        # scale all_trials 
        all_trials_scaled = np.zeros_like(all_trials)
        for cell in range(all_trials.shape[0]):
            for trial in range(all_trials.shape[1]):
                all_trials_scaled[cell][trial] = all_trials[cell][trial] - all_trials[cell][trial][0]
         
        pre_f = all_trials_scaled[:, :, :sub_obj.pre_frames]
        stim_end = sub_obj.pre_frames + sub_obj.duration_frames
        post_f = all_trials_scaled[:, :, stim_end : stim_end + sub_obj.pre_frames]
        
        all_amplitudes = np.mean(pre_f, 2) - np.mean(post_f, 2)
        t_tests = stats.ttest_rel(np.mean(pre_f, 2).transpose(), np.mean(post_f, 2).transpose())
                                  
        # calculate stimulus-triggered average trial and scale dfof to 0 for each cell
        sta = np.mean(sub_obj.all_trials[0], axis=1)
        sta_scaled = [sta[i] - sta[i][0] for i in range(sta.shape[0])]
        
        sta_scaled = np.array(sta_scaled)
        sub_obj.stas.append(sta_scaled)
        
        # calculate amplitude of response around stim
        pre_sta = sta_scaled[:, :sub_obj.pre_frames]
        post_sta = sta_scaled[:, stim_end : stim_end + sub_obj.pre_frames]

        sta_amplitudes = np.mean(post_sta, 1) - np.mean(pre_sta, 1)
        sub_obj.sta_amplitudes.append(sta_amplitudes)

ValueError: cannot reshape array of size 517408 into shape (703,91,newaxis)

In [262]:
sub_obj.n_frames

15288

In [ ]:
# backup of old cellStaProcessing

if self.stim_start_frames:
            
            #this is the key parameter for the sta, how many frames before and after the stim onset do you want to use
            self.pre_frames = int(np.ceil(self.fps*0.5)) # 500 ms pre-stim period
            self.post_frames = int(np.ceil(self.fps*3)) # 3000 ms post-stim period

            #list of cell pixel intensity values during each stim on each trial
            self.all_trials = [] # list 1 = cells, list 2 = trials, list 3 = dff vector

            # the average of every trial
            self.stas = [] # list 1 = cells, list 2 = sta vector

            self.all_amplitudes = []
            self.sta_amplitudes = []

            self.t_tests = []
            self.wilcoxons = []

            for plane in range(self.n_planes):
                
                raw = self.raw[plane][:,self.frames] 

                all_trials = [] # list 1 = cells, list 2 = trials, list 3 = dff vector

                stas = [] # list 1 = cells, list 2 = sta vector

                all_amplitudes = []
                sta_amplitudes = []

                t_tests = []
                wilcoxons = []

                #loop through each cell
                for i, unit in enumerate(raw):

                    trials = []
                    amplitudes = []
                    df = []
                    
                    # a flat list of all observations before stim occured
                    pre_obs = []
                    # a flat list of all observations after stim occured
                    post_obs = []
                    
                    for stim in self.stim_start_frames[plane]:
                        
                        # get baseline values from pre_stim
                        pre_stim_f  = unit[ stim - self.pre_frames : stim ]
                        baseline = np.mean(pre_stim_f)

                        # the whole trial and dfof using baseline
                        trial = unit[ stim - self.pre_frames : stim + self.post_frames ]
                        trial = [ ( (f-baseline) / baseline) * 100 for f in trial ] #dff calc
                        
                        if trial: #this is because of missing data in t-series, can be removed RL
                            
                            trials.append(trial)
                            
                            #calc amplitude of response        
                            pre_f = trial[ : self.pre_frames - 1]
                            pre_f = np.mean(pre_f)
                            
                            avg_post_start = self.pre_frames + ( self.duration_frames + 1 )
                            avg_post_end = avg_post_start + int(np.ceil(self.fps*0.5)) # post-stim period of 500 ms
                            
                            post_f = trial[avg_post_start : avg_post_end]
                            post_f = np.mean(post_f)
                            amplitude = post_f - pre_f
                            amplitudes.append(amplitude)
                            
                            # append to flat lists
                            pre_obs.append(pre_f)
                            post_obs.append(post_f)
                        
                    trials = np.array(trials)
                    all_trials.append(trials)
                    
                    #average amplitudes across trials
                    amplitudes = np.array(amplitudes)
                    all_amplitudes.append(amplitudes)
                    sta_amplitude = np.mean(amplitudes, axis=0)
                    sta_amplitudes.append(sta_amplitude)

                    #average across all trials
                    sta = np.mean(trials, axis=0)        
                    stas.append(sta)
                    
                    #remove nans from flat lists
                    pre_obs = [x for x in pre_obs if ~np.isnan(x)]
                    post_obs = [x for x in post_obs if ~np.isnan(x)]
                    
                    #t_test and man whit test pre and post stim (any other test could also be used here)
                    t_test = stats.ttest_rel(pre_obs, post_obs)
                    t_tests.append(t_test)
                    
#                     wilcoxon = stats.wilcoxon(pre_obs, post_obs)
#                     wilcoxons.append(wilcoxon)

                self.all_trials.append(np.array(all_trials))
                self.stas.append(np.array(stas))
                
                self.all_amplitudes.append(np.array(all_amplitudes))
                self.sta_amplitudes.append(np.array(sta_amplitudes))

                self.t_tests.append(np.array(t_tests))
                self.wilcoxons.append(np.array(wilcoxons))
            
#             plt.figure()
#             plt.plot([avg_post_start] * 2, [-1000, 1000])
#             plt.plot([avg_post_end] * 2, [-1000, 1000])
#             plt.plot([self.pre_frames - 1] * 2, [-1000, 1000])
#             plt.plot([0] * 2, [-1000, 1000])
#             plt.plot(stas[5])
#             plt.plot(stas[10])
#             plt.plot(stas[15])
#             plt.ylim([-100,200]) 

            self._staSignificance(test)
            self._singleTrialSignificance()

In [ ]:
def plotAllDff(sub_obj, threshold):
    
    #all trials (dff based on sub_obj.pre_frames pre-stim period)
    all_trials_dff = np.array(sub_obj.all_trials[0])

    #s1 cell ids
    in_s1 = np.array(sub_obj.cell_s1[0])
    s1_cells = np.where(in_s1)

    #s1 trials
    s1_trials_dff = all_trials_dff[s1_cells][:][:]
    s1_trials_dff_avg = np.mean(s1_trials_dff, axis=1)
    #distribution of s1 trial dff values (few really high/low)
#     plt.hist(np.mean(s1_trials_dff, axis=(1,2)));
#     plt.axis([-2500, 1000, 0, 5])

    #filter high/low values out
    mean_dff_s1 = np.mean(s1_trials_dff_avg, axis=1)
    extreme_s1_means = np.where(abs(mean_dff_s1) > threshold)
    print(len(extreme_s1_means[0]), 'extreme values excluded')
    s1_trials_dff_avg = np.delete(s1_trials_dff_avg, extreme_s1_means, axis=0)
    
    plt.plot(s1_trials_dff_avg.transpose(), lw=0.2);
    plt.plot(np.mean(s1_trials_dff_avg, axis=0), lw=5)
    plt.axis([0, 54, -10, 25])

In [ ]:
sub_obj = exp_obj.photostim_s
threshold = 500

plt.figure()
plotAllDff(sub_obj, threshold)

sub_obj = exp_obj.photostim_r

plt.figure()
plotAllDff(sub_obj, threshold)

sub_obj = exp_obj.spont

plt.figure()
plotAllDff(sub_obj, threshold)

### Play with whisker stim data

In [ ]:
# Calculate the number of cells that responded both to whisker stim and photostim

# a = np.where(exp_obj.whisker_stim.sta_sig[0])
# b = np.where(exp_obj.photostim_r.sta_sig[0])
# print(a,b)
# sum(1 for i in b[0] if i in a[0])

# Plot whisker stim trials where cells went over 200 dFF

plt.figure(figsize=(5,10))
sta_mean = []

for i, sta in enumerate(exp_obj.whisker_stim.stas[0]):
    if exp_obj.whisker_stim.sta_sig[0][i]:
        plt.plot(sta, alpha=0.2)
        sta_mean.append(sta)
    
        if exp_obj.whisker_stim.sta_amplitudes[0][i] > 200:
            print(exp_obj.whisker_stim.cell_id[0][i])
            
sta_mean = np.mean(sta_mean, axis=0)

plt.plot(sta_mean, 'k', linewidth=2);

plt.axis([0, 55, -10, 300]);

### Play with dataframes from master object

In [ ]:
with open(master_path, 'rb') as f:
    master_obj = pickle.load(f)

df = master_obj.df
print(df.columns)

In [ ]:
df